## QnA with 1800+ papers from ICML 2023

In this notebook, you will be able to

1. Download ThirdAI's Neural DB trained on all the papers accepted to ICML 2023.

2. Ask any question and get relevant references from any of the 1800+ papers.

3. (Optional) How to use your OpenAI key to generate grounded answers without hallucination.

To learn more about NeuralDB and it's capabilities, refer to our demo here: https://github.com/ThirdAILabs/Demos/blob/main/neural_db/main_example.ipynb


You can optionally run this notebook in Google Colab using this link: https://githubtocolab.com/ThirdAILabs/Demos/blob/main/neural_db/examples/icml_2023.ipynb

In [ ]:
!pip3 install thirdai --upgrade
!pip3 install "thirdai[neural_db]"  # You may have to unquote this
!pip3 install openai
!pip3 install paper-qa
!pip3 install langchain

In [1]:
from thirdai import licensing
import os
if "THIRDAI_KEY" in os.environ:
    licensing.activate(os.environ["THIRDAI_KEY"])
else:
    licensing.activate("")  # Enter your ThirdAI key here

In [2]:
from thirdai import neural_db as ndb

db = ndb.NeuralDB(user_id="my_user") # you can use any username, in the future, this username will let you push models to the model bazaar

In [ ]:
## Download and extract the ICML 2023 Neural DB checkpoint
## This step takes ~2 mins

import os
import zipfile

checkpoint = "./icml_2023"
if not os.path.exists(checkpoint):
    if not os.path.exists(checkpoint+".zip"):
        os.system("wget -O "+checkpoint+".zip 'https://www.dropbox.com/scl/fi/8q1yni3ncf0a82qyrzj3k/icml_2023.zip?rlkey=t47p8bu4x1l8arvycpemz2htm&dl=0'")

    with zipfile.ZipFile(checkpoint+".zip", 'r') as zip_ref:
        zip_ref.extractall(checkpoint)


In [17]:
## Load the checkpoint

db = ndb.NeuralDB.from_checkpoint('./icml_2023/')

### Let's ask!

In [14]:
search_results = db.search(query="some ideas to align memory access pattern with parameter access pattern and cache hierarchies", top_k=2)

In [15]:
for result in search_results:
    print(result.text)
    # print(result.source)
    print('************')

This paper proposes a model-agnostic cache-friendly and hardware- aware model compression approach: Random Operation Access Specific Tile (ROAST) hash- ing. ROAST collapses the parameters by club- bing them through a lightweight mapping. While clubbing these parameters ROAST utilizes cache hierarchies by aligning the memory access pattern with the parameter access pattern.
************
Separability Constant k. Our data generating distribu- tion Dk with larger k is more separable in the sense that the two Gaussian distributions overlap less and they are more likely to generate well-separated training data. This separa- bility constant k is of great importance in our analysis as we will demonstrate the curse of separability phenomenon based on the dependency of sample complexity on k.
************


### Get Answers from OpenAI using Langchain

In this section, we will show how to use LangChain and query OpenAI's QnA module to generate an answer from the references that you retrieve from the above DB. You'll have to specify your own OpenAI key for this module to work. You can replace this segment with any other generative model of your choice. You can choose to use an source model like LLAMA 2 or MPT for answer generation with the same prompt that you use with OpenAI.

In [18]:
import os
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = ""  # Enter your OpenAI key here

In [19]:
from langchain.chat_models import ChatOpenAI
from paperqa.prompts import qa_prompt
from paperqa.chains import make_chain

llm = ChatOpenAI(
    model_name='gpt-3.5-turbo', 
    temperature=0.1,
)

qa_chain = make_chain(prompt=qa_prompt, llm=llm)

In [20]:
def get_references(query):
    search_results = db.search(query,top_k=3)
    references = []
    for result in search_results:
        references.append(result.text)
    return references

def get_answer(query, references):
    return qa_chain.run(question=query, context='\n\n'.join(references[:3]), answer_length="abt 50 words")

In [21]:
query = "some ideas to align memory access pattern with parameter access pattern and cache hierarchies"

references = get_references(query)
print(references)

['This paper proposes a model-agnostic cache-friendly and hardware- aware model compression approach: Random Operation Access Specific Tile (ROAST) hash- ing. ROAST collapses the parameters by club- bing them through a lightweight mapping. While clubbing these parameters ROAST utilizes cache hierarchies by aligning the memory access pattern with the parameter access pattern.', 'Separability Constant k. Our data generating distribu- tion Dk with larger k is more separable in the sense that the two Gaussian distributions overlap less and they are more likely to generate well-separated training data. This separa- bility constant k is of great importance in our analysis as we will demonstrate the curse of separability phenomenon based on the dependency of sample complexity on k.', 'This indicates that as the number of training samples/classes increases there exists a progressively larger mismatch between the knowl- edge learned from ImageNet and the knowledge needed for distinguishing new 

In [22]:
answer = get_answer(query, references)

print(answer)

To align the memory access pattern with the parameter access pattern and cache hierarchies, one approach is to utilize a lightweight mapping technique called Random Operation Access Specific Tile (ROAST) hashing. ROAST collapses the parameters by clubbing them together, allowing for efficient memory access and utilization of cache hierarchies (Paper2019). This approach ensures that the memory access pattern is aligned with the parameter access pattern, optimizing cache utilization and improving overall performance.
